FunctionSet.py contains a set of functions that do things like take finite difference derivatives of grid variables.

Also, numpy is used here so that array adding is easier to do.

In [1]:
from FunctionSet import *
# The NRPy_param_funcs module sets up global structures that manage free parameters within NRPy+
import NRPy_param_funcs as par 
# The indexedexp module defines various functions for defining and managing indexed quantities like tensors and pseudotensors
import indexedexp as ixp
# The grid module defines various parameters related to a numerical grid or the dimensionality of indexed expressions
# For example, it declares the parameter DIM, which specifies the dimensionality of the indexed expression
import grid as gri
from outputC import *
import numpy as np

In [2]:
import sympy
from sympy import symbols, IndexedBase, Indexed, Idx, preorder_traversal

Declaring grid variables and indices. Note that the variables to be evolved are the vector potential and electric field.

In [10]:
Nx, Ny, Nz, Nn= symbols('Nx Ny Nz Nn', integer=True)
i = Idx('i', Nx)
j = Idx('j', Ny)
k = Idx('k', Nz)
n = Idx('n', Nn)
dx, dy, dz = symbols('dx dy dz')
Evec = [IndexedBase('Ex'),IndexedBase('Ey'),IndexedBase('Ez')]
Avec = [IndexedBase('Ax'),IndexedBase('Ay'),IndexedBase('Az')]
phi = IndexedBase('phi')
var_A = ['Ax','Ay','Az']
var_E = ['Ex','Ey','Ez']
var_phi = ['phi']
var_names = var_A+var_E+var_phi

Attaching grid indices to the declared variables.  

Also... 'Low' is meant to show that this variable is a vector with a lower index.   This is unimportant in flat space here, but will be necessary to keep track of when a curved metric is used in future developments.

In [11]:
ELow = ixp.zerorank1(DIM = 3)
for itr in range(3):
    ELow[itr]=Evec[itr][i,j,k,n]
    
ALow = ixp.zerorank1(DIM = 3)
for itr in range(3):
    ALow[itr]=Avec[itr][i,j,k,n]
    
phi = phi[i,j,k,n]

ELow = np.array(ELow)
ALow = np.array(ALow)

Righthand side of Maxwell evolution equations written in a symbolic form.  These are in vacuum for now, but can easily be extended to include currents.

In [12]:
RHS_dotALow = -ELow-np.array(grad(phi))
RHS_dotELow = np.array(grad(div(ALow)))-LapTen(ALow)

Making use of the AMReXcode to convert RHSs to compilable code blocks to be inserted in the AMReX advance.cpp code. You can choose to prepend the expression with the updated "new" fabs.

In [16]:
AMRout = ""
for itr in range(len(directions)):
    AMRout += AMReXcode(RHS_dotALow[itr],var_names,True,var_A[itr]) + "\n\n"
print(AMRoutA)

for itr in range(len(directions)):
    AMRout += AMReXcode(RHS_dotELow[itr],var_names,True,var_E[itr]) + "\n\n"
print(AMRoutE)

Ax_new_fab(i, j, k, n) = -Ex_old_fab(i, j, k, n) - (-phi_old_fab(i - 1, j, k, n) + phi_old_fab(i + 1, j, k, n))/(2*dx[0]);

Ay_new_fab(i, j, k, n) = -Ey_old_fab(i, j, k, n) - (-phi_old_fab(i, j - 1, k, n) + phi_old_fab(i, j + 1, k, n))/(2*dx[1]);

Az_new_fab(i, j, k, n) = -Ez_old_fab(i, j, k, n) - (-phi_old_fab(i, j, k - 1, n) + phi_old_fab(i, j, k + 1, n))/(2*dx[2]);


Ex_new_fab(i, j, k, n) = -(Ax_old_fab(i, j, k - 1, n) + Ax_old_fab(i, j, k + 1, n) - 2*Ax_old_fab(i, j, k, n))/(dx[2]*dx[2]) - (Ax_old_fab(i, j - 1, k, n) + Ax_old_fab(i, j + 1, k, n) - 2*Ax_old_fab(i, j, k, n))/(dx[1]*dx[1]) + (-(-Az_old_fab(i - 1, j, k - 1, n) + Az_old_fab(i - 1, j, k + 1, n))/(2*dx[2]) + (-Az_old_fab(i + 1, j, k - 1, n) + Az_old_fab(i + 1, j, k + 1, n))/(2*dx[2]) - (-Ay_old_fab(i - 1, j - 1, k, n) + Ay_old_fab(i - 1, j + 1, k, n))/(2*dx[1]) + (-Ay_old_fab(i + 1, j - 1, k, n) + Ay_old_fab(i + 1, j + 1, k, n))/(2*dx[1]) - (-Ax_old_fab(i - 2, j, k, n) + Ax_old_fab(i, j, k, n))/(2*dx[0]) + (Ax_old_fab(i 